In [1]:
from neo4j import GraphDatabase

In [2]:
URI = "neo4j://neo4j"
driver = GraphDatabase.driver(URI)

In [3]:
def read_tx(tx, query):
    return [record.values() for record in tx.run(query)]

In [4]:
def read(query):
    with driver.session(database="neo4j") as session:
        return session.execute_read(read_tx, query)

In [5]:
def exec_tx(tx, query):
    tx.run(query)

In [6]:
def execute(query):
    with driver.session(database="neo4j") as session:
        session.execute_write(exec_tx, query)

In [7]:
query = """DROP INDEX ftix_product_desc IF EXISTS"""
execute(query)

In [8]:
query = """CREATE FULLTEXT INDEX ftix_product_desc FOR (p:Product) ON EACH [p.desc]"""
execute(query)

In [9]:
query = """DROP INDEX ftix_product_url IF EXISTS"""
execute(query)

In [10]:
query = """
CREATE FULLTEXT INDEX ftix_product_url FOR (p:Product) ON EACH [p.url]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'url'
  }
}
"""
execute(query)

In [11]:
res = read("MATCH (n:Product) RETURN n LIMIT 3")
for r in res:
    print(r)

[<Node element_id='4:d2f580c4-f8dd-4138-8177-a677d7d52058:0' labels=frozenset({'Product'}) properties={'images': ['https://rukminim1.flixcart.com/image/128/128/kfoapow0-0/t-shirt/c/t/k/l-m7aw19cn027b-m7-by-metronaut-original-imafw2k3arnteref.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k13w4280/t-shirt/a/t/j/m-m7aw19cn027b-m7-by-metronaut-original-imafkqwbygu54tn7.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k7531jk0/t-shirt/a/t/j/xl-m7aw19cn027b-m7-by-metronaut-original-imafpgy8at5r64ac.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k13w4280/t-shirt/a/t/j/m-m7aw19cn027b-m7-by-metronaut-original-imafkqwbxe5z9zwk.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k13w4280/t-shirt/a/t/j/m-m7aw19cn027b-m7-by-metronaut-original-imafkqwbjzz2huzh.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k13w4280/t-shirt/a/t/j/m-m7aw19cn027b-m7-by-metronaut-original-imafkqwbczzcbtuw.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/k13w4280/

In [12]:
res = read(
    """
    MATCH (sc)-[r:SUBCATEGORY_OF]->(c)
    RETURN c.name AS Category, sc.name as SubCategory
    LIMIT 3
""",
)
for r in res:
    print(r)

['Clothing and Accessories', 'Clothing Accessories']
['Clothing and Accessories', 'Winter Wear']
['Clothing and Accessories', 'Raincoats']


In [13]:
res = read("MATCH (n:Seller) RETURN n.name LIMIT 3")
for r in res:
    print(r)

['WabbaJea']
['Suhani Garment']
['ReliableCart']


In [14]:
res = read("MATCH ()-[r:SELLS]->(p) WHERE r.out_of_stock RETURN p.pid LIMIT 3")
for r in res:
    print(r)

['CTPFVPZDEQYYBKCT']
['CTPFVZNX3KKES9DH']
['CTPFVPKKWBZUCH7Y']


In [15]:
res = read(
    """
    MATCH ()-[r:SELLS]->()
    RETURN min(r.selling_price)
    """,
)
for r in res:
    print(r)

[99.0]


In [16]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.selling_price > 50 AND r.selling_price < 150
    RETURN COUNT(DISTINCT p)
    """,
)
for r in res:
    print(r)

[64]


In [17]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.selling_price > 50 AND r.selling_price < 150
    RETURN p.pid
    LIMIT 3
    """,
)
for r in res:
    print(r)

['TSHFMFXGFJ7G2ABK']
['TSHFMFT7VASAHBH3']
['TSHF5FRXKGF6A4FH']


In [18]:
res = read(
    """
    MATCH (p:Product)
    RETURN DISTINCT p.brand
    LIMIT 3
    """,
)
for r in res:
    print(r)

['M7 By Metrona']
['Wab']
['A to Z life sty']


In [19]:
res = read(
    """
    MATCH
        (p:Product)-[r1:HAS_CATEGORY]->
        (sc:SubCategory)-[r2:SUBCATEGORY_OF]->
        (c:Category)
    WHERE c.name = 'Toys'
    RETURN p.pid
    LIMIT 3
    """,
)
for r in res:
    print(r)

['HATEZMCYMGPRQWVK']


In [20]:
res = read("MATCH ()-[r:SELLS]->(p) WHERE r.discount IS NOT NULL RETURN p.pid LIMIT 3")
for r in res:
    print(r)

['TKPFCZ9EHFCY5Z4Y']
['TKPFCZ9ESZZ7YWEF']
['TKPFCZ9EVXKBSUD7']


In [21]:
res = read(
    """
    MATCH ()-[r:SELLS]->(p)
    WHERE r.discount > 80
    RETURN p.pid, r.discount ORDER BY r.discount DESC
    LIMIT 3
    """,
)
for r in res:
    print(r)

['TSHF5FRXKGF6A4FH', 87.0]
['TSHFMFT7VASAHBH3', 86.0]
['TSHFMFXGFJ7G2ABK', 86.0]


In [22]:
res = read(
    """
    MATCH (p:Product)
    RETURN DISTINCT
        SUBSTRING(
            REDUCE(x = '', y IN SPLIT(p.url, '/')[0..3] | x + '/' + y),
            1
        )
    LIMIT 3
    """,
)
for r in res:
    print(r)

['https://www.flipkart.com']


In [23]:
res = read(
    """
    MATCH (s)-[r:SELLS]->(p)
    WHERE s.name = 'SOCCERINTERNATIONAL'
    RETURN p.name
    LIMIT 3
    """,
)
for r in res:
    print(r)

['Tennis Shoes For Men\xa0\xa0(Orange, Grey)']
['Cricket Shoes For Men\xa0\xa0(White, Black)']
['Football Shoes For Men\xa0\xa0(Green, Blue)']


In [24]:
res = read(
    """
    MATCH
        (p)-[r:HAS_CATEGORY]->
        (sc)-[rsc:SUBCATEGORY_OF]->
        (c)
    WHERE p.pid = 'SHTFRTGVPM9D4PZE'
    RETURN c.name, sc.name
    """,
)
for r in res:
    print(r)

['Clothing and Accessories', 'Topwear']


In [25]:
res = read("MATCH (n:Product) WHERE n.brand = 'Lev' RETURN n.pid, n.name LIMIT 3")
for r in res:
    print(r)

['SHTFTSGN2DXJYYU5', 'Men Regular Fit Printed Spread Collar Casual Shirt']
['SHTFRTH5GGX8CV7T', 'Men Slim Fit Printed Casual Shirt']
['SHTFQEENGHYUUMSJ', 'Men Slim Fit Checkered Spread Collar Casual Shirt']


In [26]:
res = read(
    """
    MATCH (p)-[r:HAS_CATEGORY]->(c)
    WHERE c.name = "Clothing Accessories"
    RETURN p.pid, p.name
    LIMIT 3
    """,
)
for r in res:
    print(r)

['CTPFVZT9ZYJB4WJZ', 'nu-Lite Satin Tie & Cufflink\xa0\xa0(Multicolor)']
['CAPFHJQ65YG6TBYV', 'Solid muslim cap cotton Cap']
['CAPE3MVWGTGMCXEP', 'Printed Woolen Cap']


In [27]:
res = read(
    """
    MATCH
        (p)-[r:HAS_CATEGORY]->
        (sc)-[rsc:SUBCATEGORY_OF]->
        (c)
    WHERE c.name = 'Footwear'
    RETURN p.pid, p.name
    LIMIT 3
    """,
)
for r in res:
    print(r)

['SHOE9B7AURZFZYS9', 'Prospeed Full Spike Cricket Shoes For Men\xa0\xa0(Multicolor)']
['SFFFUZ95MWYZVRUX', 'Slides']
['SFFFWZFUXEZENHPE', 'Slides']


In [28]:
res = read(
    """
    MATCH (s)-[r:SELLS]->(p)
    WHERE r.selling_price > 6000 and r.selling_price < 10000 and s.name <> ''
    RETURN DISTINCT s.name
    LIMIT 3
    """,
)
for r in res:
    print(r)

['ArvindTrueBlue']
['PumaSportsIndia']
['RetailNet']


In [29]:
res = read(
    """
    MATCH (n:Product)
    WHERE date(n.crawled_at) = date('2021-02-11')
    RETURN COUNT(n)
    """,
)
for r in res:
    print(r)

[8424]


In [30]:
res = read(
    """
    MATCH (n:Product)
    WHERE n.crawled_at > datetime('2021-02-11T00:00:01Z')
    RETURN COUNT(n)
    """,
)
for r in res:
    print(r)

[8422]


In [31]:
res = read(
    """
    MATCH (n:Product)
    WHERE n.crawled_at >= datetime() - duration("P30D")
    RETURN COUNT(n)
    """,
)
for r in res:
    print(r)

[0]


In [32]:
res = read(
    """
    MATCH (n:Product)
    WHERE n.crawled_at >= datetime() - duration("P5Y")
    RETURN COUNT(n)
    """,
)
for r in res:
    print(r)

[28080]


In [33]:
res = read("MATCH (n:Product) WHERE size(n.images) > 1 RETURN n.pid LIMIT 3")
for r in res:
    print(r)

['TSHFGP4FDENQDRQD']
['TSHFGXZYDNZYGSFT']
['TSHFGXZYMERZN5EZ']


In [34]:
res = read(
    """
    MATCH (s)-[r:SELLS]->(p)
    WHERE p.price > 5000
    RETURN
        p.pid,
        p.price,
        r.selling_price,
        (r.selling_price / p.price) AS percentage_from_original_price
    ORDER BY (r.selling_price / p.price)
    LIMIT 5
    """,
)
for r in res:
    print(r)

['TSHFHSWH5WJK3HJ7', 5500.0, 950.0, 0.17272727272727273]
['TSHFKNMWDGQPQ2WX', 6800.0, 1200.0, 0.17647058823529413]
['TSHFZWRU9HESS44Z', 6392.0, 1199.0, 0.18757822277847308]
['TSHFZWRTH2GHXYUM', 6392.0, 1199.0, 0.18757822277847308]
['TSHFJJNQXXWZYFKM', 5488.0, 1100.0, 0.20043731778425655]


In [35]:
res = read(
    """
    MATCH (n:Product)
    WHERE n.url CONTAINS 'otracker=browse'
    RETURN n.pid, n.url
    LIMIT 3
    """,
)
for r in res:
    print(r)

['TSHFGP4FDENQDRQD', 'https://www.flipkart.com/m7-metronaut-graphic-print-men-round-neck-grey-t-shirt/p/itm5deb1d231cdb1?pid=TSHFGP4FDENQDRQD&lid=LSTTSHFGP4FDENQDRQDH4H75B&marketplace=FLIPKART&srno=b_4_154&otracker=browse&fm=organic&iid=1c224076-7064-491c-8215-f017a0d6079a.TSHFGP4FDENQDRQD.SEARCH&ssid=wdnznn0mnk0000001612113601726']
['TSHFGXZYDNZYGSFT', 'https://www.flipkart.com/m7-metronaut-printed-men-round-neck-white-black-t-shirt/p/itmdfa508bb6b350?pid=TSHFGXZYDNZYGSFT&lid=LSTTSHFGXZYDNZYGSFTUKPIA2&marketplace=FLIPKART&srno=b_4_155&otracker=browse&fm=organic&iid=1c224076-7064-491c-8215-f017a0d6079a.TSHFGXZYDNZYGSFT.SEARCH&ssid=wdnznn0mnk0000001612113601726']
['TSHFGXZYMERZN5EZ', 'https://www.flipkart.com/m7-metronaut-color-block-men-round-neck-dark-blue-blue-t-shirt/p/itm30fcf1ffacafb?pid=TSHFGXZYMERZN5EZ&lid=LSTTSHFGXZYMERZN5EZUBGMQA&marketplace=FLIPKART&srno=b_4_156&otracker=browse&fm=organic&iid=1c224076-7064-491c-8215-f017a0d6079a.TSHFGXZYMERZN5EZ.SEARCH&ssid=wdnznn0mnk00000016

In [36]:
res = read(
    """
    CALL
        db.index.fulltext.queryNodes(
            "ftix_product_url",
            '"otracker=browse"'
        ) YIELD node, score
    RETURN node.pid, node.url, score
    LIMIT 3
    """,
)
for r in res:
    print(r)

['DHTFZSY2YEJ8XXGH', 'https://www.flipkart.com/vairavaa-tex-2021-kerala-traditional-mundu-1-inch-blue-silver-fancy-kasavu-border-100-cotton-4-00-meter-double-solid-men-dhoti/p/itm3647c1ca42061?pid=DHTFZSY2YEJ8XXGH&lid=LSTDHTFZSY2YEJ8XXGHOHCFDC&marketplace=FLIPKART&srno=b_1_18&otracker=browse&fm=organic&iid=456873da-97fd-4d94-b7f3-7225acc45d84.DHTFZSY2YEJ8XXGH.SEARCH&ssid=lbddbx574g0000001612414112897', 3.9408187866210938]
['DHTFZSY278HAJGVK', 'https://www.flipkart.com/vairavaa-tex-2021-kerala-traditional-mundu-1-inch-black-silver-fancy-kasavu-border-100-cotton-4-00-meter-double-solid-men-dhoti/p/itmd933ccffe3595?pid=DHTFZSY278HAJGVK&lid=LSTDHTFZSY278HAJGVKFXJAOR&marketplace=FLIPKART&srno=b_1_28&otracker=browse&fm=organic&iid=456873da-97fd-4d94-b7f3-7225acc45d84.DHTFZSY278HAJGVK.SEARCH&ssid=lbddbx574g0000001612414112897', 3.9408187866210938]
['HKFFSF8Z45FFJ2RV', 'https://www.flipkart.com/royal-mart-6-piece-white-15-inch-complete-face-cover-handkerchief-men-s-cotton-striped-comfortable-c

In [37]:
res = read(
    """
    MATCH (n:Product)
    WHERE LOWER(n.desc) CONTAINS 'goku'
    RETURN n.pid, n.desc
    LIMIT 3
    """,
)
for r in res:
    print(r)

['TSHFYF4ZF7BXTKRV', 'Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!']


In [38]:
res = read(
    """
    CALL
        db.index.fulltext.queryNodes(
            "ftix_product_desc",
            '"goku"'
        ) YIELD node, score
    RETURN node.pid, node.desc, score
    """,
)
for r in res:
    print(r)

['TSHFYF4ZF7BXTKRV', 'Dragon ball goku gohan Premium Quality Round Neck T-Shirts With 100% Combed Cotton Bio Washed Fabric Which Is Gentaly Tailored & Designed For A Premium And High Quality Shopping Experince. It Is Proudly Designed In INDIA By WILDSTER CLOTHINGS PVT LTD. Happy Shopping !!', 4.5496931076049805]


In [39]:
res = read(
    """
    MATCH (s)-[r:SELLS]->(p)
    RETURN s.name, round(avg(p.avg_rating), 3)
    LIMIT 3
    """,
)
for r in res:
    print(r)

['Shyam Enterprises', 3.75]
['SH ENTERPRISE', 4.2]
['NextEdgeRetails', 2.853]


In [40]:
res = read(
    """
    MATCH (p)-[rc:HAS_CATEGORY]->(sc:SubCategory)-[rs:SUBCATEGORY_OF]->(c:Category)
    RETURN c.name, sc.name, COUNT(p) AS product_count
    LIMIT 5
    """,
)
for r in res:
    print(r)

['Toys', 'Party Supplies', 1]
['Clothing and Accessories', 'Uber Urban Clothing and Accessories', 28]
['Clothing and Accessories', 'Roy Clothing and Accessories', 12]
['Clothing and Accessories', 'INSPIRE Clothing and Accessories', 4]
['Clothing and Accessories', 'Inspire Clothing and Accessories', 23]


In [41]:
res = read(
    """
    MATCH (p)-[rc:HAS_CATEGORY]->(sc:SubCategory)-[rs:SUBCATEGORY_OF]->(c:Category)
    RETURN c.name, COUNT(DISTINCT p) AS product_count
    LIMIT 5
    """,
)
for r in res:
    print(r)

['Toys', 1]
['Clothing and Accessories', 27118]
['Bags, Wallets & Belts', 41]
['Footwear', 920]


In [42]:
res = read(
    """
    MATCH (n:Product)
    RETURN n.brand, COUNT(n) AS product_count
    LIMIT 3
    """,
)
for r in res:
    print(r)

['M7 By Metrona', 255]
['Wab', 53]
['A to Z life sty', 1]


In [43]:
res = read("MATCH ()-[r:SELLS]->() RETURN avg(r.discount)")
for r in res:
    print(r)

[50.25660141760606]


In [44]:
res = read(
    """
    MATCH (s)-[r:SELLS]->()
    WHERE r.discount IS NOT NULL
    RETURN s.name, avg(r.discount)
    ORDER BY avg(r.discount) DESC
    LIMIT 3
    """,
)
for r in res:
    print(r)

['Jackson Consolidate', 81.8]
['FASLA PRODUCTS PVT LTD', 80.4]
['Dagcros Merchandising', 80.0]


In [45]:
res = read(
    """
    MATCH (p:Product)
    WHERE p.brand IS NOT NULL AND p.brand <> ''
    RETURN p.brand, COUNT(p)
    ORDER BY COUNT(p) DESC
    LIMIT 5
    """,
)
for r in res:
    print(r)

['ECKO Unl', 951]
['Free Authori', 860]
['ARBO', 806]
['REEB', 802]
['Pu', 798]


In [46]:
res = read(
    """
    MATCH (s)-[r:SELLS]->(p:Product)
    WITH count(DISTINCT p) AS total_product_count
    MATCH (s)-[r:SELLS]->(p)
    RETURN
        r.out_of_stock,
        COUNT(p),
        total_product_count,
        floor(COUNT(p))/total_product_count AS percentage
    """,
)
for r in res:
    print(r)

[False, 26441, 28080, 0.9416310541310541]
[True, 1644, 28080, 0.05854700854700855]


In [47]:
res = read(
    """
    MATCH
        (s:Seller)-[r:SELLS]->
        (p:Product)-[rc:HAS_CATEGORY]->
        (sc:SubCategory)-[rs:SUBCATEGORY_OF]->
        (c:Category)
    RETURN
        c.name,
        count(p) AS product_count,
        round(avg(r.selling_price), 3) AS average_selling_price
    LIMIT 5
    """,
)
for r in res:
    print(r)

['Toys', 1, 189.0]
['Clothing and Accessories', 27123, 713.348]
['Bags, Wallets & Belts', 41, 258.512]
['Footwear', 920, 502.067]


In [48]:
res = read(
    """
    MATCH
        (s:Seller)-[r:SELLS]->
        (p:Product)-[rc:HAS_CATEGORY]->
        (sc:SubCategory)-[rs:SUBCATEGORY_OF]->
        (c:Category)
    RETURN
        c.name,
        sc.name,
        count(p) AS product_count,
        round(avg(r.selling_price), 3) AS average_selling_price
    LIMIT 5
    """,
)
for r in res:
    print(r)

['Toys', 'Party Supplies', 1, 189.0]
['Clothing and Accessories', 'Uber Urban Clothing and Accessories', 28, 1265.036]
['Clothing and Accessories', 'Roy Clothing and Accessories', 12, 500.333]
['Clothing and Accessories', 'INSPIRE Clothing and Accessories', 4, 499.0]
['Clothing and Accessories', 'Inspire Clothing and Accessories', 23, 831.174]


In [49]:
res = read(
    """
    MATCH
        (s:Seller)-[r:SELLS]->(p:Product)
    RETURN
        p.crawled_at.year,
        p.crawled_at.month,
        p.crawled_at.day,
        p.crawled_at.hour,
        round(avg(r.selling_price), 3) AS average_selling_price
    ORDER BY
        p.crawled_at.year,
        p.crawled_at.month,
        p.crawled_at.day,
        p.crawled_at.hour
    LIMIT 3
    """,
)
for r in res:
    print(r)

[2021, 2, 10, 20, 757.935]
[2021, 2, 10, 21, 741.746]
[2021, 2, 10, 22, 773.68]
